## Crawl and Scrape all info from the category: drugs

In [1]:
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

socks.set_default_proxy(socks.SOCKS5, "localhost", 9050) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

# ToRReZ market URL DEZE LINKE GEBRUIKEN OM DE COOCKIES TE PAKKEN
url = "http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=1"


# Manually selected from the browser header after solving the captcha
cookies_copied = 'Cookie: ray_id=9bc0af157bce9cafa0a2b005a3ddc26d141c4aa7308e2e496c1cfbb089a07031; hr4ujvby8ds459og4kzcpbzwjdj_session=eyJpdiI6IjBIYVYzcWhrcng0dXFQazNaYlJxV2c9PSIsInZhbHVlIjoiWm9WOXJ6N3l0bUkxaXVyUGNcLzBpSXFrQ3VCMEFkNDRaYUplS2d6OVd5a09TOFdENm96UzFNdFoxdE9FYkFxa1wvTWVldkRqMktVT29JNWxIQUlaRmNpWnpPc0RWTXFUSEZJTnd6TFU5N2Nja1dVSXUxUXdTZTRVZ2V4RnJ2UW1pbSIsIm1hYyI6IjIzZjFlN2Y0MWE4MjRjMzIyN2UwZDQxMGRhNmFlOGJmODFjZmEwMDUzMTFjYTJlYTdjM2E3YmJiMjRlODM5NWUifQ%3D%3D; XSRF-TOKEN=eyJpdiI6ImxWelVNWUFiK3pJTE9iUitvRHRySUE9PSIsInZhbHVlIjoiaXhoMXloNVNFbU9pN3NrTWNQb0xKbkJibWtBMnNxdHFoYWJ4MkRhcmRWMjI2RUJkbWpwMGV0cFNLSGUxcDdVVENRbUdCSXBENm13SGlsUmRBTEJ0WloxaU1adnZlakxVdnVtSHo0WTR3cHV3N0Z1ZTBzZk02SFRxYlV5SkRBQ2QiLCJtYWMiOiJmMTVlMzMyZTkxNWZkNzJhOTNhNWQ3YzlhNjA1YzU2NmE1NmQzNDFlYzMzMDE3NmY3ZjkyMjdiYTc5YmUxNzlkIn0%3D'


# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    return cookies_set


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
soup = BeautifulSoup(web_page.content, 'html.parser')



# create empty lists
vendor = []    
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a',attrs={"class" : "page-link"}):
    pages.append(a.text)
last_page = pages[-2]

for page in range(1, int(last_page)+1): #int(last_page)+1

    # NIET AANPASSEN
    url = 'http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')


    for a in soup.find_all('a', href=True, class_='title'):
        all_titles.append(a.text.strip())

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])
        
    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        elif len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        elif len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())

           
    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingFrom') != None:
            ship_from.append(tag_elm.find('span', class_='shippingFrom').text.strip())
        else:
            ship_from.append(np.nan)

    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingTo') != None:
            ship_to.append(tag_elm.find('span', class_='shippingTo').text.strip())
        else:
            ship_to.append(np.nan)

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())


            
            
results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles, 
                    all_prices, ship_from, ship_to)))



df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3', 
                                   'highest_category', 'product', 'price in $', 'shipping_from', 'shipping_to'])    


In [2]:
df

,vendor,category_level_1,category_level_2,category_level_3,highest_category,product,price in $,shipping_from,shipping_to
0,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1 Ounce (28 grams) Penis Envy Magic Mushrooms,400.00,United States,United States
1,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1/4 lb (4 oz) Golden Teacher Magic Mushrooms,600.00,United States,United States
2,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1 Oz Smll Capsules Golden Teachers Magic Mushroom,180.00,United States,United States
3,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1/2 Ounce (14 grams) Penis Envy Magic Mushrooms,210.00,United States,United States
4,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1 Ounce (28 grams) Golden Teacher Magic Mushr...,185.00,United States,United States
5,ShroomTown,Drugs and Chemicals,Psychedelics,Mushrooms,Mushrooms,1/2 Oz Lrge Capsles Golden Teacher Magic Mush...,100.00,United States,United States
6,wedoweed,Drugs and Chemicals,Cannabis & Hash,Vaping,Vaping,THC Vape Juice (Sativa) 30ml (Free Shipping),85.68,Australia,Australia
7,wedoweed,Drugs and Chemicals,Cannabis & Hash,NaN,Cannabis & Hash,1g 60% Shatter Concentrate (Free Shipping),46.74,Australia,Australia
8,wedoweed,Drugs and Chemicals,Cannabis & Hash,Edibles,Edibles,Mct Tincture 30ml (mild effects) (Free Shipping),35.05,Australia,Australia
9,CornerShop,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,"Lemon Haze liquid e cigartette, E Zigarette",24.07,Germany,Europe


## Crawl and Scrape all vendor information

In [3]:
unique_vendors = list(dict.fromkeys(vendor))

vendor_page = []

for vendor in unique_vendors:
    vendor_page.append("http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/profile/" + str(vendor))

since = [] 
transactions = []
feedback_total = []
feedback_positive = []
feedback_negative = []
disputes_total =[]
disputes_won = []
disputes_lost = []
finalize_early = []
rank = []
verification = []

all_variables = [since, transactions, feedback_total, feedback_positive, 
                 feedback_negative, disputes_total, disputes_won, disputes_lost, finalize_early]
all_text = ['On ToRReZ Market Since:', 'Total Amount Of Transactions', 'Total Feedback Received', 
            'Positive Feedback Received Ratio', 'Negative Feedback Received Ratio', 'Disputes Total:', 
            'Disputes Won:', 'Disputes Lost:', 'Finalize Early']    
    
for page in vendor_page:
    web_page = s.get(page, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')
    
    for numb in range(0,len(all_variables)):
        for i in soup.find_all('table', class_='table table-borderless table-sm mt-1 table-fixed'):
            value = soup.find('th', text=all_text[numb])
            all_variables[numb].append(value.find_next_sibling('td').text.strip())

    for span in soup.find_all('span', class_='badge badge-success question'):
        rank.append(span.text.strip())
        
    for tag_elm in soup.find_all('div', class_='col singleItemDetails mt-2'):
        if tag_elm.find('a', class_='badge badge-dark') != None:
            verification.append(tag_elm.find('a', class_='badge badge-dark').text.strip())
        else:
            verification.append(np.nan)


In [4]:
results_2 = (list(zip(unique_vendors, rank, verification, since, transactions, feedback_total, feedback_positive, feedback_negative, 
                      disputes_total, disputes_won, disputes_lost, finalize_early)))



df_2 = pd.DataFrame(results_2,columns=['vendor', 'rank', 'verifcation', 'since', 'transactions', 'feedback_total', 
                                   'feedback_positive', 'feedback_negative','disputes_total',
                                   'disputes_won', 'disputes_lost', 'finalize_early'])    


In [5]:
df_2

,vendor,rank,verifcation,since,transactions,feedback_total,feedback_positive,feedback_negative,disputes_total,disputes_won,disputes_lost,finalize_early
0,ShroomTown,Rank 1,Verification Level 10,"Aug 4, 2020",72,30,100% (30),0% (0),1,0% (0),0% (0),Available (only optional)
1,wedoweed,Rank 2,Verification Level 2,"Oct 18, 2020",159,78,100% (78),0% (0),0,0% (0),0% (0),Not Available
2,CornerShop,Rank 0,NaN,"Jan 31, 2021",9,1,100% (1),0% (0),1,0% (0),0% (0),Not Available
3,GQDS24,Rank 2,Verification Level 3,"Oct 31, 2020",115,85,100% (85),0% (0),4,0% (0),25% (1),Not Available
